In [1]:
import pandas as pd
import numpy as np
import os
import time

## load date

In [2]:
df_07 = pd.read_csv('SPTCC-20160701.csv', encoding='gbk', header = None)
df_08 = pd.read_csv('SPTCC-20160802.csv', encoding='gbk', header = None)
df_09 = pd.read_csv('SPTCC-20160901.csv', encoding='gbk', header = None)

columns = ['卡ID', '刷卡日期' , '刷卡时间', '刷卡站点', '刷卡乘车类型', '刷卡扣钱', '是否优惠']
df_07.columns = columns
df_08.columns = columns
df_09.columns = columns

df = pd.concat([df_07,df_08,df_09])

del df_07,df_08,df_09

df = df.query("刷卡乘车类型 == '地铁'") # (29816659, 7)

In [3]:
stations = set(df['刷卡站点'])

## features
### 是否进站

In [4]:
df['是否进站'] = df['刷卡扣钱'].apply(lambda x : 1 if x > 0 else 0)

### datetime

In [6]:
df['DateTime'] = df['刷卡日期'] + " " + df["刷卡时间"]
df['DateTime'] = pd.to_datetime(df['DateTime'])
df['月份'] = df["DateTime"].dt.month
df['日'] = df["DateTime"].dt.day
df['时'] = df["DateTime"].dt.hour
df['分'] = df["DateTime"].dt.minute
df['星期序数'] = df["DateTime"].dt.dayofweek
df['是否周末'] = df['星期序数'].apply(lambda x : 1 if x >=5 else 0)

df['第x个五分钟'] = df['分'].apply(lambda x : x // 5)
df['进出站状态'] = df['是否进站'].replace(0,-1)

df.to_csv('df.csv',index = None )

## 重新读取数据

In [7]:
df = pd.read_csv('df.csv')

df['月份'] = df['月份'].astype(str)
df['日'] = df['日'].astype(str)
df['时'] = df['时'].astype(str)
df['第x个五分钟'] = df['第x个五分钟'].astype(str)


### 进站 - 出站 = 站内人流

In [12]:
data = df.groupby(['刷卡站点','月份', '日', '时', '第x个五分钟' ])['进出站状态'].sum().to_frame()
data .to_csv('data.csv', encoding='gbk')
data = data.reset_index()
data['月份'] = data['月份'].astype(int)
data['日'] = data['日'].astype(int)
data['时'] = data['时'].astype(int)
data['第x个五分钟'] = data['第x个五分钟'].astype(int)



### 进站 + 出战 = 站内人流

In [9]:
count_df = df.groupby(['刷卡站点','月份', '日', '时', '第x个五分钟' ])['进出站状态'].count().to_frame()
count_df .to_csv('count_df.csv', encoding='gbk')